In [1]:
from datetime import datetime, date
import os

import pandas as pd
import pyspark 
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import pyspark.pandas as ps

spark = SparkSession.builder.appName("Practice").getOrCreate()

22/06/20 14:35:55 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
22/06/20 14:35:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 14:35:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.read.csv('dummy_schedule_data.csv', header=True, sep=",")
df2 = spark.read.csv('dummy_schedule_data2.csv', header=True, sep=",")

df_union = df1.union(df2)

df_union.show(truncate=False)

+----------+---------------------+---------------------+-----------------+-----------+
|start_time|away_team            |home_team            |date             |proper_date|
+----------+---------------------+---------------------+-----------------+-----------+
|7:00p     |Dallas Mavericks     |Golden State Warriors|Tue, May 16, 2022|2022-05-16 |
|7:00p     |Golden State Warriors|Dallas Mavericks     |Tue, May 16, 2022|2022-05-16 |
|7:30p     |Dallas Mavericks     |Golden State Warriors|Tue, May 16, 2022|2022-05-16 |
|7:30p     |Golden State Warriors|Dallas Mavericks     |Tue, May 16, 2022|2022-05-16 |
|7:30p     |Miami Heat           |Boston Celtics       |Tue, May 23, 2022|2022-05-23 |
|8:30p     |FakeGame1            |FakeGame2            |Tue, May 23, 2022|2022-05-23 |
+----------+---------------------+---------------------+-----------------+-----------+



In [41]:
schema_1 = df_union.schema[0].dataType
schema_2 = df_union.schema['start_time'].dataType
schema_3 = df_union.schema

print(df_union.printSchema())
# df_union.dtypes
print(schema_3)

root
 |-- start_time: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- date: string (nullable = true)
 |-- proper_date: string (nullable = true)

None
StructType(List(StructField(start_time,StringType,true),StructField(away_team,StringType,true),StructField(home_team,StringType,true),StructField(date,StringType,true),StructField(proper_date,StringType,true)))


In [8]:
# these are all the same

df_union.filter("start_time == '7:30p'").show()

df_union.filter(df_union.start_time == '7:30p').show()

df_union.where(df_union.start_time == '7:30p').show()

df_union.where(df_union['start_time'] == '7:30p').show()

df_union.where(df_union['start_time'] == F.lit('7:30p')).show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 1

In [9]:
groupby1 = df_union.groupBy(df_union.start_time).count()

print(groupby1.head(5))

groupby2 = df_union \
    .filter(df_union.start_time == '7:30p') \
    .groupBy(df_union.start_time) \
    .count()

print(groupby2.head(5))

groupby_df = groupby2.toPandas()

[Row(start_time='7:00p', count=2), Row(start_time='7:30p', count=3), Row(start_time='8:30p', count=1)]
[Row(start_time='7:30p', count=3)]


In [44]:
df_union.where(df_union['start_time'] == '7:30p').show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [45]:
df_union.where(df_union['start_time'] == F.lit('7:30p')).show()

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [10]:
df_union.createOrReplaceTempView("test_table")
df = spark.table("test_table")

In [60]:
dd = spark.sql("select * from test_table where start_time = '7:30p';").toPandas() # pulls the data into a pandas a dataframe
dd2 = spark.sql("select * from test_table where start_time = '7:30p';").collect() # pulls the data into a nested python list
spark.sql("select * from test_table where start_time = '7:30p';").show() # prints it out into the notebook

+----------+--------------------+--------------------+-----------------+-----------+
|start_time|           away_team|           home_team|             date|proper_date|
+----------+--------------------+--------------------+-----------------+-----------+
|     7:30p|    Dallas Mavericks|Golden State Warr...|Tue, May 16, 2022| 2022-05-16|
|     7:30p|Golden State Warr...|    Dallas Mavericks|Tue, May 16, 2022| 2022-05-16|
|     7:30p|          Miami Heat|      Boston Celtics|Tue, May 23, 2022| 2022-05-23|
+----------+--------------------+--------------------+-----------------+-----------+



In [11]:
# being able to read plans helps you to predict potential bottlenecks in your larger queries.
# spark keeps a whole bunch of info about this test_table, which came from df_union, which came from 2 separate csv files i loaded in.
# project means the columns that will be projected (or selected).
# hashaggregate means data aggregation (with group by).
# this prints out the parsed, analyzed, and optimized logical plans and then the physical plan.

# the physical plan is what gets ran on the executors
spark.sql("select start_time, count(*) as num_records from test_table where start_time = '7:30p' group by start_time;").explain(extended = True)

== Parsed Logical Plan ==
'Aggregate ['start_time], ['start_time, 'count(1) AS num_records#392]
+- 'Filter ('start_time = 7:30p)
   +- 'UnresolvedRelation [test_table], [], false

== Analyzed Logical Plan ==
start_time: string, num_records: bigint
Aggregate [start_time#127], [start_time#127, count(1) AS num_records#392L]
+- Filter (start_time#127 = 7:30p)
   +- SubqueryAlias test_table
      +- View (`test_table`, [start_time#127,away_team#128,home_team#129,date#130,proper_date#131])
         +- Union false, false
            :- Relation [start_time#127,away_team#128,home_team#129,date#130,proper_date#131] csv
            +- Relation [start_time#153,away_team#154,home_team#155,date#156,proper_date#157] csv

== Optimized Logical Plan ==
Aggregate [start_time#127], [start_time#127, count(1) AS num_records#392L]
+- Union false, false
   :- Project [start_time#127]
   :  +- Filter (isnotnull(start_time#127) AND (start_time#127 = 7:30p))
   :     +- Relation [start_time#127,away_team#128,ho

In [17]:
groupby2 = df_union \
    .select(F.col('start_time'), F.col('away_team'), F.col('home_team'), F.col('proper_date')) \
    .filter(df_union.start_time == '7:30p') \
    .groupBy(df_union.start_time) \
    .count() \
    .explain(extended = True)
    # .count()
# just about the same plan as the spark sql one.

== Parsed Logical Plan ==
Aggregate [start_time#127], [start_time#127, count(1) AS count#441L]
+- Filter (start_time#127 = 7:30p)
   +- Project [start_time#127, away_team#128, home_team#129, proper_date#131]
      +- Union false, false
         :- Relation [start_time#127,away_team#128,home_team#129,date#130,proper_date#131] csv
         +- Relation [start_time#153,away_team#154,home_team#155,date#156,proper_date#157] csv

== Analyzed Logical Plan ==
start_time: string, count: bigint
Aggregate [start_time#127], [start_time#127, count(1) AS count#441L]
+- Filter (start_time#127 = 7:30p)
   +- Project [start_time#127, away_team#128, home_team#129, proper_date#131]
      +- Union false, false
         :- Relation [start_time#127,away_team#128,home_team#129,date#130,proper_date#131] csv
         +- Relation [start_time#153,away_team#154,home_team#155,date#156,proper_date#157] csv

== Optimized Logical Plan ==
Aggregate [start_time#127], [start_time#127, count(1) AS count#441L]
+- Union fal

In [7]:
df_union2 = df_union \
    .groupBy(F.col('start_time')) \
    .agg(F.count('start_time').alias('count_start_times')) \
    .show()

+----------+-----------------+
|start_time|count_start_times|
+----------+-----------------+
|     7:00p|                2|
|     7:30p|                3|
|     8:30p|                1|
+----------+-----------------+



In [77]:
# creating window functions like in sql which are basically named aggregates w/ a group by

# creating new columns using pyspark functions for date / timestamp, and case when statement.

# you can either do .filter(F.col('colname') == xxx) syntax or do .filter(df.colname == xxx) syntax
df_union3 = df_union \
    .withColumn("count_start_times", F.count("start_time").over(Window.partitionBy('start_time'))) \
    .withColumn("count_start_times_sum", F.sum("count_start_times").over(Window.partitionBy('start_time'))) \
    .withColumn("count_start_times_avg", F.avg("count_start_times").over(Window.partitionBy('start_time'))) \
    .withColumn('proper_date_plus1', F.date_add(df_union.proper_date, 1)) \
    .withColumn('scrape_date', F.current_date()) \
    .withColumn('scrape_ts', F.current_timestamp()) \
    .withColumn('scrape_year', F.year(F.current_timestamp())) \
    .withColumn('scrape_month', F.month(F.current_timestamp())) \
    .withColumn('case_when_test', F.when(F.col('home_team') == 'Golden State Warriors', 'WOOT') \
                                  .when(F.col('home_team') == 'Dallas Mavericks', 'OOF') \
                                  .otherwise('CRAP')) \
    .filter(F.col('start_time').isin(['7:00p', '7:30p'])) # these 2 are the same
    # .filter((F.col('start_time') == '7:00p') | (F.col('start_time') == '7:30p'))

df_union3.select('scrape_ts').show(1)

df_pandas = df_union3.toPandas()

+--------------------+
|           scrape_ts|
+--------------------+
|2022-06-15 13:14:...|
+--------------------+
only showing top 1 row



In [131]:
import numpy as np
# pandas way of doing the above.
df1 = pd.read_csv('dummy_schedule_data.csv')
df2 = pd.read_csv('dummy_schedule_data2.csv')

df_union = df1.merge(df2, how = 'outer')

df_union['count_start_times'] = df_union \
    .groupby('start_time')['start_time'] \
    .transform('count')

df_union['case_when_test'] = np.where(df_union['home_team'] == 'Golden State Warriors', 'WOOT',
                             np.where(df_union['home_team'] == 'Dallas Mavericks', 'OOF',
                             'CRAP'))